# Use Custom Image, Software Specification and Runtime Definition to deploy a python function with `ibm-watson-machine-learning`

This notebook demonstrates how to deploy in Watson Machine Learning service a python function which requires to create custom software specification and runtime definition.  
Familiarity with oc and python is helpful. This notebook uses Python 3.9.


## Learning goals

The learning goals of this notebook are:

-  Creating a Custom Image
-  Creating custom software specification and runtime definition
-  Online deployment of python function
-  Scoring data using deployed function

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.  [Create custom image](#image)
4.	[Create software specification](#soft)
3.	[Create runtime definition](#runtime) 
4.	[Web service creation and scoring](#deploy)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
password = 'PASTE YOUR PASSWORD HERE'
url = 'PASTE THE PLATFORM URL HERE'
bedrock_url = 'PASTE THE PLATFORM BEDROCK URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.0'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="image"></a>
## 2. Create a Custom Image

After the creation of a Custom image as described in https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=deployments-deploying-using-custom-image. We need to create a new base software specification for the custom image and the runtime definition for it. This notebook shows how to perform these steps and use the custom image in a WML deployment:
1. Create a new base software specification for the custom image
2. Create the new runtime definition file for the custom image
3. Create and Deploy a WML `python function` model with the custom image



_Required Role: You must be a Cloud Pak for Data cluster administrator_ 


The Creation of the custom image involves the following steps, which cannot be done from a notebook. It involves pulling building pushing docker images and can be performed in the cluster.
1. Downloading the base image https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=image-downloading-base
2. Creating and uploading a custom image https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=image-creating-custom



#### 1. Downloading the base image


To Find out which CPD release you are on.
```
    oc get wmlbase wml-cr -o=jsonpath='{.spec.version}'
```

Select the base image by refering to the documentation. Base image selected for this example:
```
    wml-deployment-runtime-py38-1:4.0.2.78-amd64
```

For this example the WML cluster installation uses the private container registry `cp.stg.icr.io` with the namespace `/cp/cpd/` and so, the base image will be available from:
```
    cp.stg.icr.io/cp/cpd/wml-deployment-runtime-py38-1:4.0.2.78-amd64
```


If you do not remember which private registry have been enabled, you can use the below queries and links to help in identifying. These registrys are configured at installation time. more info - https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=tasks-configuring-your-cluster-pull-images
```
    oc get imageContentSourcePolicy/mirror-config -o yaml    
    oc get secret/pull-secret -n openshift-config -o jsonpath='{.data.\.dockerconfigjson}' | base64 -d
```

Pull the base image:
```
    podman login -u <user> -p <password> cp.stg.icr.io/cp/cpd/
    podman pull cp.stg.icr.io/cp/cpd/wml-deployment-runtime-py38-1:4.0.2.78-amd64
```


#### 2. Creating and uploading a custom image

##### 2.1 Building the custom image
Dockerfile sample for this example is as below. We are going to install the program `gdb` as well as the python package `simplejson` here.

```
ARG  base_image_tag
FROM ${base_image_tag}

# For installing OS packages, use root:root
# e.g.
USER root:root
RUN yum install -y gdb

# For installing to conda site-packages,
# use wmlfuser:condausers
# e.g.
USER wmlfuser:condausers
RUN umask 002 && \
   pip install simplejson

```

Build the custom image `wml-demo-image:test-1`:
```
    podman build -t wml-demo-image:test-1 \
        --build-arg base_image_tag=cp.stg.icr.io/cp/cpd/wml-deployment-runtime-py38-1:4.0.2.78-amd64 \
        -f Dockerfile
```

Some sanity checks on image for this example.
```
    podman run -it wml-demo-image:test-1 bash
    $ gdb --version
    $ conda list | grep simplejson
    $ uwsgi --version
    $ exit
```

##### 2.2 Uploading the custom image

We have the following Options described below on where to upload the custom image. In this demo, we are going to use the internal registry (Option 1).


_Option 1:_ Push the image to internal registry.

Check if default route is enabled
```
    oc get configs.imageregistry.operator.openshift.io/cluster -o=jsonpath='{.spec.defaultRoute}
```

Get default route URL and login
```
    DEFAULT_ROUTE_REG=$(oc get route default-route -n openshift-image-registry -o=jsonpath='{ .spec.host }')
    echo $DEFAULT_ROUTE_REG
    
    e.g. default-route-openshift-image-registry.apps.wml402devdep3.cp.fyre.ibm.com
    podman login -u kubeadmin -p $(oc whoami -t) --tls-verify=false $DEFAULT_ROUTE_REG
```
Get internal registry namespace of WML
```
    WML_REG_NAMESPACE=$(oc get wmlbase wml-cr -o=jsonpath='{.metadata.namespace}')
    echo $WML_REG_NAMESPACE
```  
Tag and push the custom image to the `default route` registry
```
    podman tag wml-demo-image:test-1 $DEFAULT_ROUTE_REG/$WML_REG_NAMESPACE/wml-demo-image:test-1
    podman push $DEFAULT_ROUTE_REG/$WML_REG_NAMESPACE/wml-demo-image:test-1 --tls-verify=false
```

Required custom image:
```
    echo image-registry.openshift-image-registry.svc:5000/$WML_REG_NAMESPACE/wml-demo-image:test-1

    e.g.
    image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1
```

In this demo, the custom image was built in a machine which is part of the cluster. If image is built in a machine outside of the cluster, then additional Openshift related setups may be required to be able to push to the internal registry. Help on Openshift registry - https://docs.openshift.com/container-platform/4.8/registry/securing-exposing-registry.html

_Option 2: If you know the private container registry. just tag and push it there._
```
    e.g
    podman login -u <user> -p <password> cp.stg.icr.io/cp/cpd/
    podman tag wml-demo-image:test-1 cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
    podman push cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
    
    Required custom image:
    cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
```



### Credentials and generating tokens

In [1]:
import requests, json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [1]:

CPDHOST=url
BEDROCK_URL=bedrock_url
PASSWORD=password
USERNAME=username

# namespace here is the wmlbase namespace:
#    oc get wmlbase wml-cr -o=jsonpath='{.metadata.namespace}'
namespace="zen"

## Generate Token

In [5]:
cpd_token_resp = !curl -sSk -X POST \
    -H "Content-Type: application/x-www-form-urlencoded;charset=UTF-8" \
    -d "grant_type=password&username={USERNAME}&password={PASSWORD}&scope=openid" \
    {BEDROCK_URL}/idprovider/v1/auth/identitytoken

cpd_token = json.loads(cpd_token_resp[-1])["access_token"]

In [6]:
token_ep = CPDHOST + "/v1/preauth/validateAuth"
token_auth_resp = requests.get(token_ep,
                               headers={"username": USERNAME,
                                        "iam-token": cpd_token},
                               verify=False)
token = token_auth_resp.json()["accessToken"]

In [7]:
headers={'Authorization': 'Bearer ' + token,
         'Content-Type': 'application/json'}
VERSION = '2020-07-08'

## Create a Volume in an existing PVC - "cc-home-pvc"

Creation of Base software specification and Runtime definition require uploading of the metadata json files to certain PVC path. We will use Volume Service APIs to access the PVC `cc-home-pvc`

In [8]:
# name for this Vol Service instance
display_name='CCHome'

In [9]:
response = requests.post(
    CPDHOST + '/zen-data/v3/service_instances',
    headers=headers,
    json={
        "addon_type": "volumes",
        "addon_version": "-",
        "create_arguments": {
            "metadata": {
                "existing_pvc_name": "cc-home-pvc"
            }
        },
        "namespace": namespace,
        "display_name": display_name
    },
    verify=False)
response.status_code, response.json()

(200, {'id': '1635916789402144'})

In [10]:
vol_svc_id=response.json()['id']
vol_svc_id

'1635916789402144'

### Check Volume Instances - if it is already created

In [11]:
response = requests.get(CPDHOST + '/zen-data/v3/service_instances',
                        headers=headers,
                        verify=False)
response.status_code

200

In [13]:
vol_svc_id = None
for svc_inst in response.json()['service_instances']:
    print(svc_inst['display_name'], svc_inst['id'], svc_inst['metadata'].get('existing_pvc_name'))
    if svc_inst['display_name'] == display_name:
        vol_svc_id = svc_inst['id']

        
print('\nvol_svc_id:', vol_svc_id)

CCHome 1635916789402144 cc-home-pvc
deploy-fvt-zen-ready 1630415688762284 volumes-deploy-fvt-zen-pvc
deploy-fvt-zen 1630415685720759 None

vol_svc_id: 1635916789402144


### Check the new volume instance

In [14]:
response = requests.get(CPDHOST + f'/zen-data/v3/service_instances/{vol_svc_id}',
                        headers=headers,
                        verify=False)
print(json.dumps(response.json(), indent=2))

{
  "service_instance": {
    "addon_type": "volumes",
    "addon_version": "-",
    "created_at": "2021-11-03T05:19:49.489000Z",
    "display_name": "CCHome",
    "id": "1635916789402144",
    "instance_identifiers": null,
    "metadata": {
      "existing_pvc_name": "cc-home-pvc"
    },
    "misc_data": {},
    "namespace": "zen",
    "owner_uid": "1000330999",
    "owner_username": "admin",
    "parameters": {},
    "provision_status": "PROVISIONED",
    "resources": {},
    "roles": [
      "Admin"
    ],
    "updated_at": "2021-11-03T05:19:49.481394Z",
    "zen_service_instance_info": {
      "docker_registry_prefix": "quay.io/opencloudio"
    }
  }
}


### Start file server on volume

In [15]:
response = requests.post(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_messageCode_": "200",
  "message": "Successfully started volume service"
}


In [36]:
# In case you want to restart
response = requests.delete(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{display_name}',
    headers=headers,
    json={},
    verify=False)
# print(json.dumps(response.json(), indent=2))
response = requests.post(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_messageCode_": "200",
  "message": "Successfully started volume service"
}


<a id='soft'></a>
# 3. Create a new base software specification for the custom image

We will create the base software specification `cspec-38-1-t1` for this demo

In [16]:
sw_spec_name='cspec-38-1-t1'
sw_spec_filename=f'{sw_spec_name}.json'
print('software spec name     :', sw_spec_name)
print('software spec filename :', sw_spec_filename)

software spec name     : cspec-38-1-t1
software spec filename : cspec-38-1-t1.json


####  Custom Image software specification:
- `entity.software_specification.type` have to be set to "base" to indicate custom image.
- `entity.software_specification.built_in` have to be set boolean `False` to indicate custom image.
- `entity.software_configuration.included_packages` array is informational, it can be empty list `[]` . 
- `entity.software_configuration.platform.name` should be set to `python` .

In [17]:
sw_spec = {
  "metadata": {
    "name": sw_spec_name,
    "description": "Test"
  },
  "entity": {
    "software_specification": {
      "type": "base",
      "built_in": False,
      "package_extensions": [],
      "display_name": f"Test {sw_spec_name}",
      "software_configuration": {
        "included_packages": [
          {
            "name": "simplejson",
            "version": "3.17.5"
          },
          {
            "name": "dummy-b",
            "version": "0.0.1"
          }         
        ],
        "platform": {
          "name": "python",
          "version": "3.8"
        }
      }
    }
  }
}

### Upload file to the specified path on the volume

In [18]:
target_file=f'%2F_global_%2Fconfig%2Fenvironments%2Fsoftware-specifications%2F{sw_spec_filename}'
target_file

'%2F_global_%2Fconfig%2Fenvironments%2Fsoftware-specifications%2Fcspec-38-1-t1.json'

In [19]:
# multipart/form-data
# Do not set the Content-type header yourself, leave that to pyrequests to generate

response = requests.put(
    CPDHOST + f'/zen-volumes/{display_name}/v1/volumes/files/{target_file}',
    headers={
        "Authorization": f"Bearer {token}",
    },
    files={
        'upFile': (sw_spec_filename, json.dumps(sw_spec), 'application/json')
    },
    verify=False)
response.status_code

200

In [20]:
response.json()

{'_messageCode_': 'Success',
 'message': 'Successfully uploaded file and created the directory structure'}

### We can check if the file was successfully upload

In [21]:
response = requests.get(CPDHOST + f'/zen-volumes/{display_name}/v1/volumes/files/{target_file}',
                        headers=headers,
                        verify=False)
print(response.status_code, '\n', json.dumps(response.json(), indent=2))

200 
 {
  "metadata": {
    "name": "cspec-38-1-t1",
    "description": "Test"
  },
  "entity": {
    "software_specification": {
      "type": "base",
      "built_in": false,
      "package_extensions": [],
      "display_name": "Test cspec-38-1-t1",
      "software_configuration": {
        "included_packages": [
          {
            "name": "simplejson",
            "version": "3.17.5"
          },
          {
            "name": "dummy-b",
            "version": "0.0.1"
          }
        ],
        "platform": {
          "name": "python",
          "version": "3.8"
        }
      }
    }
  }
}


### Wait for about 5 mins
Refresh intervel is handled by `env-spec-sync-job` cronjob
```
e.g.
oc get cronjobs env-spec-sync-job

NAME                SCHEDULE      SUSPEND   ACTIVE   LAST SCHEDULE   AGE
env-spec-sync-job   */5 * * * *   False     0        82s             23d
```

### Check if the new Software Specification is activated

In [25]:
response = requests.get(
    CPDHOST + f'/v2/software_specifications?name={sw_spec_name}',
    headers=headers,
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "total_results": 1,
  "resources": [
    {
      "metadata": {
        "created_at": "2021-11-03T05:25:04.075Z",
        "updated_at": "2021-11-03T05:25:04.075Z",
        "name": "cspec-38-1-t1",
        "description": "Test",
        "asset_type": "software_specification",
        "href": "/v2/software_specifications/abe090f5-92fc-5945-81f0-27136b56306f",
        "asset_id": "abe090f5-92fc-5945-81f0-27136b56306f"
      },
      "entity": {
        "software_specification": {
          "type": "base",
          "built_in": false,
          "package_extensions": [],
          "display_name": "Test cspec-38-1-t1",
          "software_configuration": {
            "included_packages": [
              {
                "name": "simplejson",
                "version": "3.17.5"
              },
              {
                "name": "dummy-b",
                "version": "0.0.1"
              }
            ],
            "platform": {
              "name": "python",
              "versio

### **Warning**: Do not proceed unless the new software specification is activated!

<a id="runtime"></a>
# 4. Create the new runtime definition file for the custom image

We will create the Runtime definition file, which connects the software specification `cspec-38-1-t1` to the custom image name `image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1`

In [26]:
# fully qualified name for the Internal Registry image of custom image
custom_image_fqn='image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1'
custom_image_fqn

'image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1'

#### Runtime Definition file for Custom image:
- filename needs to end with `*-server.json`
- the prefix in filename `<runtime_definition_name>-server.json` will be the runtime definition name. If `name` is provided, `name` will be ignored.
- `runtimeType` should be `wml` for custom image
- This runtime definition is the link between the software specification and the image name for custom image
- If the custom image had been uploaded to the private container registry, then the value for `custom_image_fqn` would be `cp.stg.icr.io/cp/cpd/wml-demo-image:test-1`

In [27]:
runtime_definition={
  "displayName": f"Test {sw_spec_name}",
  "description": "Test description",
  "author": "IBM",
  "tested": True,
  "isService": True,
  "features": [
    "wml"
  ],
  "runtimeType": "wml",
  "software_specification_name": sw_spec_name,
  "image": custom_image_fqn
}
print(json.dumps(runtime_definition, indent=2))

{
  "displayName": "Test cspec-38-1-t1",
  "description": "Test description",
  "author": "IBM",
  "tested": true,
  "isService": true,
  "features": [
    "wml"
  ],
  "runtimeType": "wml",
  "software_specification_name": "cspec-38-1-t1",
  "image": "image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1"
}


### Upload file to the specified path on the volume¶

In [28]:
runtime_definition_filename=f'{sw_spec_name}-server.json'
runtime_definition_filename

'cspec-38-1-t1-server.json'

In [29]:
# multipart/form-data
# Do not set the Content-type header yourself, leave that to pyrequests to generate

response = requests.put(
    CPDHOST + f'/zen-data/v1/volumes/files/%2F_global_%2Fconfig%2F.runtime-definitions%2Fibm',
    headers={
        "Authorization": f"Bearer {token}",
    },
    files={
        'upFile': (runtime_definition_filename, json.dumps(runtime_definition), 'application/json')
    },
    verify=False)
response.status_code

200

In [30]:
response.json()

{'_messageCode_': 'Success',
 'message': 'Successfully uploaded file and created the directory structure'}

### Check upload runtime definition file.

In [31]:
response = requests.get(
    CPDHOST + 
    f'/zen-data/v1/volumes/files/%2F_global_%2Fconfig%2F.runtime-definitions%2Fibm%2F{runtime_definition_filename}',
    headers=headers,
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "displayName": "Test cspec-38-1-t1",
  "description": "Test description",
  "author": "IBM",
  "tested": true,
  "isService": true,
  "features": [
    "wml"
  ],
  "runtimeType": "wml",
  "software_specification_name": "cspec-38-1-t1",
  "image": "image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1"
}


### Check if the new Runtime Definition is activated

In [32]:
response = requests.get(
    CPDHOST + f'/v2/runtime_definitions', 
    headers=headers,
    verify=False)
response.status_code

200

In [33]:
# since we have named our runtime defination filename as <sw_spec>-server.json
# we can look for that name

run_def_name=sw_spec_name

for r in response.json()['resources']:
    if r['entity']['runtime_type'] == 'wml' and r['entity']['name'] == run_def_name:
        print(json.dumps(r, indent=2))
        
# The `entity.software_configuration` metadata needs special authorization to see details.
# It will be shown as empty dict here (internally it is not necessarily empty).

{
  "metadata": {
    "guid": "fc69bfed-af1f-453c-ac57-8bec9bfd336e",
    "created_at": "2021-11-03T05:25:50.339Z"
  },
  "entity": {
    "name": "cspec-38-1-t1",
    "display_name": "Test cspec-38-1-t1",
    "description": "Test description",
    "author": "IBM",
    "tested": true,
    "is_service": true,
    "features": [
      "wml"
    ],
    "runtime_type": "wml",
    "software_configuration": {}
  }
}


In [34]:
print('The required software specification name:', sw_spec_name)
print('\t', custom_image_fqn)

The required software specification name: cspec-38-1-t1
	 image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1


### Stop File server on the Volume

In [35]:
response = requests.delete(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_messageCode_": "200",
  "message": "Successfully stopped volume service"
}


### Cleanup - delete the Volume instance

In [36]:
response = requests.delete(CPDHOST + f'/zen-data/v3/service_instances/{vol_svc_id}',
                           headers=headers,
                           verify=False)
print(json.dumps(response.json(), indent=2))

{
  "id": "1635916789402144"
}


<a id="deploy"></a>

## 5. Create and Deploy a WML `python function` model with the custom image

The demo `python function` below showcase the package `simplejson` which was installed for this custom image is now usable by returning the `version`, and does nothing else useful beyond that.

In [40]:
%%writefile aifun_{sw_spec_name}.py

def score_generator():
        
    # Define scoring function
    def callModel(payload_scoring):
        import simplejson

        v4_scoring_response = {
            'predictions': [{'values': [simplejson.__version__]}]
        }
    
        return v4_scoring_response

    def score(input):

        # Score using the pre-defined model
        prediction = callModel(input);
        return prediction
    
    return score


score = score_generator()

Writing aifun_cspec-38-1-t1.py


`python function` needs to be gzipped for storing in WML repository

In [41]:
!gzip -fk aifun_{sw_spec_name}.py

In [43]:
!ls aifun_{sw_spec_name}*

aifun_cspec-38-1-t1.py    aifun_cspec-38-1-t1.py.gz


**We need use the Base software specification that was created earlier `cspec-38-1-t1`**

In [48]:
sw_spec_name, client.software_specifications.get_id_by_name(sw_spec_name)

('cspec-38-1-t1', 'abe090f5-92fc-5945-81f0-27136b56306f')

In [44]:
meta_props = {
   client.repository.FunctionMetaNames.NAME: "py-function for custom image t1",
   client.repository.FunctionMetaNames.DESCRIPTION: "Test Custom Image",
   client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID:
        client.software_specifications.get_id_by_name(sw_spec_name)
}
stored_function_details = client.repository.store_function(f'aifun_{sw_spec_name}.py.gz', meta_props)
stored_function_details

{'entity': {'software_spec': {'id': 'abe090f5-92fc-5945-81f0-27136b56306f',
   'name': 'cspec-38-1-t1'},
  'type': 'python'},
 'metadata': {'created_at': '2021-11-03T05:28:54.251Z',
  'description': 'Test Custom Image',
  'id': '2fbedbff-abcb-4f83-ac11-058ac03fa725',
  'modified_at': '2021-11-03T05:28:54.251Z',
  'name': 'py-function for custom image t1',
  'owner': '1000330999',
  'space_id': 'af104b1b-14e0-4ca9-86d2-08cc4c9eb141'},
 'system': {'warnings': []}}

In [46]:
function_id=client.repository.get_function_id(stored_function_details)
function_id

'2fbedbff-abcb-4f83-ac11-058ac03fa725'

### Create an online deployment

In [49]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "function dep Online t1",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC : {
        "id":  client.hardware_specifications.get_id_by_name('S')}
}
deployment_details = client.deployments.create(function_id, meta_props)



#######################################################################################

Synchronous deployment creation for uid: '2fbedbff-abcb-4f83-ac11-058ac03fa725' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..........................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b2b0a470-35f3-4316-a900-53dc80fdbfa1'
------------------------------------------------------------------------------------------------




You can monitor the cluster for pods with name prefix "wml-" and "ci-" in its name
example:
```
$ oc get pods | grep wml | grep ci-
wml-dep-od-ai-function0.0-cl-ci-wbiku1vw-764974dfd6-bwrn2       0/1     Running     0          4m58s

$ oc describe pod wml-dep-od-ai-function0.0-cl-ci-wbiku1vw-764974dfd6-bwrn2 | grep Image: | grep "wml-demo-image:test-1"
    Image:         image-registry.openshift-image-registry.svc:5000/zen/wml-demo-image:test-1
```

In [51]:
deployment_id=client.deployments.get_uid(deployment_details)
deployment_id

'b2b0a470-35f3-4316-a900-53dc80fdbfa1'

### Predict using created deployment

In [53]:
scoring_payload = {
    'input_data': [
        { 'id': 'id1', 'fields': ['dummy', 'dummy_c2'], 'values': [[1, 1]]}
    ]
}

In [54]:
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

{'predictions': [{'values': ['3.17.5']}]}

<a id="cleanup"></a>
## 6. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for function deployment and scoring with custom custom_image, software_spec and runtime definition.  
 Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics) for more samples, tutorials, documentation, how-tos, and blog posts. 

#### References:
- https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=deployments-deploying-using-custom-image
- https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=resources-volumes-api

### Author

**Ginbiaksang Naulak** Software Engineer at IBM.

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.